In [1]:
# python libraries
import numpy as np
from IPython.display import HTML, display

# pydrake imports
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    LogVectorOutput,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)

from underactuated import running_as_notebook

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Exercise Goals

In this exercise, you would learn the following:

- writing URDF to create increasingly complex multibody systems, and

- writing an LQR controller for the different variants of the cart-pole system.

By the end of this exercise you would have learned to build your system from scratch, learning about urdfs and Drake along the way.  Drake also has a tutorial on this topic.

URDF (Unified Robot Description Format) is one of the most widely used formats to describe the geometry of robots. They are represented in XML and can be stored in strings as we will do later. In this problem, we will build a double pendulum cartpole using the single pendulum cartpole described in [Section 3.2](http://underactuated.csail.mit.edu/acrobot.html#cart_pole) of the textbook as a base to understand the construction of a basic URDF then modify the system into a double pendulum cartpole. We will then wire up an LQR controller and simulate the cart-pole and perform a new series of balancing tasks.

**Note**: For the sake of this problem, we consider x to be the horizontal direction and z to be the vertical direction. 

The single pendulum cart-pole system that we are using for learning urdfs is shown below.
<img src="https://raw.githubusercontent.com/RussTedrake/underactuated/master/figures/exercises/single-pendulum-cart-pole.png" alt="Alternative text" width="500" class="center"/>


## URDFs
The simple URDFs we will be make today consist of three major components:
1.  **Links**: inertial and visual information for each link
2.  **Joints**: the connection between links
3.  **Transmissions**: control inputs to joints

We will go over each of these three in detail next.

Referring to [cartpole.urdf](https://github.com/RussTedrake/underactuated/blob/master/underactuated/models/cartpole.urdf) (used in textbook examples) can be helpful for this exercise.

### Links
A link component here has three parts: the name (used to identify the link), the inertial (used to define the mass and  center of mass of the link), and the visual (used for displaying representative images, but does not affect the system dynamics). Another component often used in urdfs is the collision geometry, which is not discussed here. You can refer to http://wiki.ros.org/urdf/XML/link for more details.



#### 1. Base Link
Below you will find the first link, which will represent the base cart. Note the three components from before: (1) the link name ("base"), (2) the inertial, with COM at (0, 0, 0) and mass of 1, and (3) the visuals, consisting of a box for the body of the cart and two spheres for the wheels with their positions set relative to the link's origin. Take a few minutes to verify these yourself.

In [3]:
# DO NOT MODIFY
base_urdf = """
  <link name="base">

    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 0" />
      <geometry>
        <box size=".5 .2 .2" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz=".15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    
    <visual>
      <origin xyz="-.15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
  </link>
"""

#### 2. Pendulum Link
Next we have the pendulum link. The pendulum is defined by a ball mass and a cylinder rod where the COM is at the ball mass. The origin for this pendulum is the center of the base (cart), that is the origin. 

In the next cell, complete the description for a pendulum that has a 5 cm radius spherical ball with mass 1 kg and a 1 m cylindrical rod of radius 1 cm. 

In [4]:
pendulum_link = """ 
  <link name="pendulum0">
    
  <!-- This is how you can write comments in urdfs -->
  <!-- TODO: Write the inertial component below -->
  <inertial>
    <origin xyz="0 0 -1"/>
    <mass value="1"/>
  </inertial>

  <!-- TODO: Write the visual component for the sphere (radius=0.05, mass=1.) below -->
  <visual>
    <origin xyz="0 0 -1"/>
    <geometry>
      <sphere radius="0.05"/>
    </geometry>
    <material>
      <color rgba="0 0 1 1"/>
    </material>
  </visual>

  <!-- TODO: Write the visual component for the cylindrical rod (radius=0.01, length=1.) below -->
  <visual>
    <origin xyz="0 0 -.5"/>
    <geometry>
      <cylinder length="1" radius=".01" />
    </geometry>
    <material>
      <color rgba="0 0 1 1"/>
    </material>
  </visual>
  

  </link>
"""

### Joints
Now we move onto joints, which are used to connect links and define their behavior. Every joint has a unique name and a type specified. Of the various type of joints, we will be using two: **prismatic** and **continuous**. You can refer to http://wiki.ros.org/urdf/XML/joint for information on the joint types available and other elements in the joints description.

#### 1. Base Joint
The first joint we must consider is how the cart fits into the world, which is the root parent of our cart-pole system. Here we treat this as a sliding or prismatic joint as if the cart is moving on a fixed track in the x direction. Observe how all this information is contained in the joint description below. The `axis` element of a joint depends on the type of the joint; for prismatic joint, it is the axis of translation, in the joint's frame.

In [5]:
# DO NOT MODIFY
base_joint = """
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="base" />
    <axis xyz="1 0 0" />
  </joint>
"""

#### 2. Pendulum Joint
The second joint to consider is for pendulum links connected to the cart base. We will treat these as continuous joints allowing them to revolve around their parent origin. Be careful about the axis of rotation: in this exercise, Y-axis is going into the page but $\theta$ points along $[0, -1, 0]$ by the right hand rule (curl your fingers in positive $\theta$, the direction of the thumb is the direction of theta).

In [6]:
pendulum_joint = """

<!-- TODO: write the parent, child, axis and origin for the pendulum joint named "theta0" with type "continuous". -->
<joint name="theta0" type="continuous">
    <parent link="base" />
    <child link="pendulum0" />
    <axis xyz = "0 1 0 "/>
    <origin xyz="0 0 0"/>
</joint>
"""

### Transmissions
Lastly, we come to the transmission component. Here the only controlled input is a force applied in the x direction on the cart base. 

In [7]:
# DO NOT MODIFY
transmission = """
  <transmission type="SimpleTransmission" name="base_force">
    <actuator name="force" />
    <joint name="x" />
  </transmission>
"""

## Assembling the URDF

Now we have all the components necessary to construct a URDF for the cart-pole. Below is a function that assembles these components and adds a header and an ender to wrap them into one system. Take a few minutes to see how the generated urdf looks like.

In [8]:
single_pendulum_urdf = f"""
<?xml version="1.0"?><robot name="OurCartPole">
{base_urdf}
{pendulum_link}
{base_joint}
{pendulum_joint}
{transmission}
  </robot>
</xml>
"""
print(single_pendulum_urdf)


<?xml version="1.0"?><robot name="OurCartPole">

  <link name="base">

    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 0" />
      <geometry>
        <box size=".5 .2 .2" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz=".15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    
    <visual>
      <origin xyz="-.15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
  </link>

 
  <link name="pendulum0">
    
  <!-- This is how you can write comments in urdfs -->
  <!-- TODO: Write the inertial component below -->
  <inertial>
    <origin xyz="0 0 -1"/>
    <mass value="1"/>
  </inertial>



## Visualizing the Cartpole URDF
Let us now visualize the generated urdf using `ModelVisualizer`. Using the controls that appear in the top right corner of the Meshcat window, change the values of $x$ and $\theta_0$ to see the cartpole in different configurations. Looks great!

In [9]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(single_pendulum_urdf, "urdf")
visualizer.Run(loop_once=not running_as_notebook)
meshcat.Delete()
meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


What is the state at the unstable equilibrium for the single-pendulum cart-pole (when the pendulum is standing upright)?
**Note**: The state here is defined by $[x, \theta_0, \dot{x}, \dot{\theta}_0]$

In [10]:
x_star_single_pendulum = [0, np.pi, 0, 0]  # TODO: modify here

## Double Pendulum URDF

Now that we've managed to create the URDF for the single pendulum cart-pole, we have all the tools to make a multi-pendulum cart-pole.  

The crucial point to note is the frame used to define links and joints: in the **child** link, all the origins of inertial and visual elements are defined wrt to the **joint** frame, and the **joint origin** is defined wrt to the frame of the **parent** link. The figure [here](http://wiki.ros.org/urdf/XML/joint) can be useful for understanding this point. 

For example, In the single pendulum case, the base-pendulum joint was the same as the parent (base) origin, and the pendulum's home state $\theta = 0$ was defined wrt to the joint (the origin). 

For the case of the double pendulum cart-pole, first think about where the pendulum$0$-pendulum$1$ joint will be located (in the frame of pendulum0). Then in the frame of this joint, think about how the second pendulum (pendulum1) wil be situated for the $\theta_1=0$ configuration. The following diagrams show the choice of $\theta_0$ and $\theta_1$, and have $x = 0$.

<img src="https://raw.githubusercontent.com/RussTedrake/underactuated/master/figures/exercises/double-pendulum-cart-pole.png" alt="Alternative text" width="700" class="center"/>


In the next cell, define the second pendulum (with same inertial and visual properties as the first pendulum), and the joint connecting the first and the second pendulum. Assemble the components for the double-pendulum cart-pole: concatenate the strings and add header and ender to generate the urdf. 

In [11]:
second_pendulum_joint = """
  <joint name="theta1" type="continuous">
    <parent link="pendulum0" />
    <child link="pendulum1" />
    <axis xyz = "0 1 0 "/>
    <origin xyz="0 0 -1"/>
  </joint>
"""

second_pendulum_link = """
  <link name="pendulum1">
    <inertial>
      <origin xyz="0 0 -1"/>
      <mass value="1"/>
    </inertial>

    <visual>
      <origin xyz="0 0 1"/>
      <geometry>
        <sphere radius="0.05"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1"/>
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 0.5"/>
      <geometry>
        <cylinder length="1" radius=".01" />
      </geometry>
      <material>
        <color rgba="0 0 1 1"/>
      </material>
    </visual>
  

  </link>
"""

# single_pendulum_urdf = f"""
# <?xml version="1.0"?><robot name="OurCartPole">
# {base_urdf}
# {pendulum_link}
# {base_joint}
# {pendulum_joint}
# {transmission}
#   </robot>
# </xml>
# """
# print(single_pendulum_urdf)

double_pendulum_urdf = f"""
<?xml version="1.0"?><robot name="doubleCartPole">
{base_urdf}
{pendulum_link}
{second_pendulum_link}
{base_joint}
{pendulum_joint}
{second_pendulum_joint}
{transmission}
  </robot>
</xml>
"""
print(double_pendulum_urdf)


<?xml version="1.0"?><robot name="doubleCartPole">

  <link name="base">

    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 0" />
      <geometry>
        <box size=".5 .2 .2" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz=".15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    
    <visual>
      <origin xyz="-.15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
  </link>

 
  <link name="pendulum0">
    
  <!-- This is how you can write comments in urdfs -->
  <!-- TODO: Write the inertial component below -->
  <inertial>
    <origin xyz="0 0 -1"/>
    <mass value="1"/>
  </inertial

Time to visualize the double-pendulum cart-pole! Observe how the two pendulums overlap with each other initally. Try varying the values for $\theta_0$ and $\theta_1$.


In [12]:
if double_pendulum_urdf != "":
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().AddModelsFromString(double_pendulum_urdf, "urdf")
    visualizer.Run(loop_once=not running_as_notebook)
    meshcat.Delete()
    meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


What is the state at the unstable equilibrium for the double-pendulum cart-pole, when both the pendulums are standing upright one over another?
**Note**: The state here is defined by $[x, \theta_0, \theta_1, \dot{x}, \dot{\theta}_0, \dot{\theta}_1]$

In [13]:
x_star_double_pendulum = [0, np.pi, np.pi, 0, 0, 0]  # TODO: modify here

## LQR for Cart-Poles
Now that we've successfully constructed a URDF file, we can add it to LQR and run our robot! 
Example 3.5 in the textbook will be helpful for building an LQR controller. Read the example carefully and fill in the following function.

In [14]:
def cartpole_balancing(cartpole_urdf, x_star, Q, R):
    """
    Arguments:
        cartpole_urdf: str
            a string that contains a urdf description of the system
        x_star: array
            a fixed point of the system; the desired equilibrium point
            we want the system to reach
        Q: state cost matrix in LQR
        R: input cost matrix in LQR

    simulates and generates an animation for the lqr controlled system
    """

    builder = DiagramBuilder()

    cartpole, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    Parser(cartpole).AddModelsFromString(cartpole_urdf, "urdf")
    cartpole.Finalize()
    cartpole.set_name("cartpole")

    
    # TODO:
    # 1. obtain the context from the cartpole plant
    
    context = cartpole.CreateDefaultContext()
    # 2. set the state vector in the context to x_star
    context.SetContinuousState(x_star)
    # 3. fix the actuation input port to zero
    cartpole.get_actuation_input_port().FixValue(context, [0])

    # 4. synthesize a LinearQuadraticRegulator, and add it to the builder
    def BalancingLQR(plant):
        # Design an LQR controller for stabilizing the CartPole around the upright.
        # Returns a (static) AffineSystem that implements the controller (in
        # the original CartPole coordinates).

        context = plant.CreateDefaultContext()
        plant.get_actuation_input_port().FixValue(context, [0])

        plant.SetPositionsAndVelocities(context, x_star)

        # MultibodyPlant has many (optional) input ports, so we must pass the
        # input_port_index to LQR.
        return LinearQuadraticRegulator(
            plant,
            context,
            Q,
            R,
            input_port_index=plant.get_actuation_input_port().get_index(),
        )

    
    # 5. wire cart-pole and lqr
    controller = builder.AddSystem(BalancingLQR(cartpole))
    builder.Connect(
        cartpole.get_state_output_port(), controller.get_input_port(0)
    )
    builder.Connect(
        controller.get_output_port(0), cartpole.get_actuation_input_port()
    )
    # add a visualizer
    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    visualizer.set_name("visualizer")

    # finish building the block diagram
    diagram = builder.Build()
    return diagram

In [15]:
def simulate_and_animate(diagram, x0, sim_time=5.0, visualize=True):
    # instantiate a simulator
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)  # makes sim faster
    simulator.get_mutable_integrator().set_fixed_step_mode(True)

    # start recording the video for the animation of the simulation
    visualizer = diagram.GetSubsystemByName("visualizer")
    visualizer.StartRecording(False)

    if (
        len(x0)
        != diagram.GetSubsystemByName("cartpole").num_continuous_states()
    ):
        print(f"Your plant doesn't have {len(x0)} state variables.")
        return

    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.0)
    context.SetContinuousState(x0)

    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)

    # stop video
    visualizer.PublishRecording()
    visualizer.DeleteRecording()

### LQR for single-pendulum cart-pole

In [16]:
# simulate and animate the lqr controller for single-pendulum cart-pole
Q = np.diag((10.0, 10.0, 1.0, 1.0))
R = np.eye(1)
x0 = np.array([0, 0.9 * np.pi, 0, 0])
single_cartpole_diagram = cartpole_balancing(
    single_pendulum_urdf, x_star_single_pendulum, Q, R
)
simulate_and_animate(
    single_cartpole_diagram,
    x0,
    sim_time=5 if running_as_notebook else 0.1,
    visualize=running_as_notebook,
)

### LQR for double-pendulum cart-pole

In [17]:
# simulate and animate the lqr controller for double-pendulum cart-pole
Q = np.diag((10.0, 10.0, 10.0, 1.0, 1.0, 1.0))
R = np.eye(1)
x0 = np.array([-2, 0.96 * np.pi, 0.93 * np.pi, 0, 0, 0])

if double_pendulum_urdf != "":
    double_cartpole_diagram = cartpole_balancing(
        double_pendulum_urdf, x_star_double_pendulum, Q, R
    )
    simulate_and_animate(
        double_cartpole_diagram,
        x0,
        sim_time=5 if running_as_notebook else 0.1,
        visualize=running_as_notebook,
    )

## Autograding
You can check your work by running the next cell. 


In [18]:
from underactuated.exercises.acrobot.test_cartpoles_urdf import (
    TestCartPolesURDF,
)
from underactuated.exercises.grader import Grader

Grader.grade_output([TestCartPolesURDF], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 2/12.

Score for Test double-pendulum cart-pole LQR states is 0/2.
- Test Failed: 
Arrays are not almost equal to 4 decimals
the dynamics for the double pendulum LQR do not match
Mismatched elements: 12 / 30 (40%)
Max absolute difference: 985.49121863
Max relative difference: 2.
 x: array([[-1.0000e-02,  3.1416e-02,  0.0000e+00,  2.3422e+01, -2.4667e+01,
         3.0692e+01],
       [ 1.0000e-01,  0.0000e+00, -6.2832e-01,  8.3137e+01, -6.6753e+01,...
 y: array([[-1.0000e-02,  3.1416e-02,  0.0000e+00, -4.6897e+02, -4.6473e+02,
         5.2840e+02],
       [ 1.0000e-01,  0.0000e+00, -6.2832e-01, -9.0235e+02, -7.7774e+02,...


Score for Test double-pendulum cart-pole dynamics is 0/3.
- Test Failed: 
Arrays are not almost equal to 6 decimals
the dynamics for the double pendulum do not match
Mismatched elements: 9 / 30 (30%)
Max absolute difference: 18.91263508
Max relative difference: 2.92740879
 x: array([[-1.000000e-02,  3.141593e-02,  0.000000e+00,  1.479371e+00,
        

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ef372e02-465c-45cf-a608-6bcfdbd7bc9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>